# Developing, Deploying and Testing Micro-services using Chalice on Jupyter Notebooks

## Hello Planet Example

In this notebook we will demonstrate more advanced topics such as:
* Environment Variables, 
* External Libraries,
* Lambda Layers, and
* Configuration Editing

## Function Setup


In [1]:
%cd ~

/home/ec2-user


In [2]:
function_name = 'hello-planet'

In [3]:
!chalice new-project $function_name

Directory already exists: hello-planet
Aborted!


In [4]:
%cd $function_name

/home/ec2-user/hello-planet


## Notebook Execution

One of the best benefits of Jupyter notebooks is interactivity. The output of each cell is presented right after and it, and it allows quick development and feedback loops. 

To allow the creation of the files for the deployment and the functions in the notebook we will define a new cell magic that both write to file and execute the code in the notebook kernel. 

In [5]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

## Passing Environment Variables

We will set some of the paramters as environment paramters, that will later be part of the environment variable of the Lambda function

In [21]:
import os

planet = 'EARTH'
os.environ['planet'] = planet

## Define the Chalice App

We will use the new magic that we created to write the file, and load the environment variable 

In [7]:
%%write_and_run app.py
from chalice import Chalice

app = Chalice(app_name='hello-planet')

import os
planet = os.environ["planet"]

## Define a helper function

We will add a simple function that is loading a table from the web (nasa.com) with planets information, and lookup the mean temperature on the given planet. This way we can provide the same functionality to the people on Mars or Venus, as long as the Lambda has access to the Internet and to NASA website. 

This function is using the magic function that both write to the app.py file and execute it in the notebook kernel. We can test its functionality directly here in the notebook.

In [8]:
%%write_and_run -a app.py
url = 'https://nssdc.gsfc.nasa.gov/planetary/factsheet/'

import pandas as pd

def get_planet_temperature(planet):
    return (
        pd
        .read_html(
            url,
            header=0,
            index_col=0
        )
        [0]
        .loc['Mean Temperature (C)',planet]
    )

### Test the helper function

In [9]:
get_planet_temperature(planet), get_planet_temperature('VENUS'), get_planet_temperature('MARS')

('15', '464', '-65')

## Define the main function

Using the function decoration `@app.route()` we define the main function that will be executed when calling the API.

In [10]:
%%write_and_run -a app.py

@app.route('/')
def index():
    return {'hello': f'{planet} with Mean Temperature (C) of {get_planet_temperature(planet)}'}

### Testing the main function

This test is done locally here in the notebook for a quick feedback and development loops. 

In [11]:
index()

{'hello': 'EARTH with Mean Temperature (C) of 15'}

## Updating external libraries

Our helper function is using `Pandas` to read the table data for the temperature lookup. The `read_html` function requires another library `lxml` to parse the HTML correctly. Chalice is using the libraries that are defined in the `requirements.txt` file, and in the next cell we will write these two libraries to the file.

In [12]:
%%writefile requirements.txt
pandas
lxml

Overwriting requirements.txt


## Updating Chalise Configurations

We will use a similar idea to add and modify the configuration files of Chalise:
* First, we read the default file that was created above with the `new-project` command. 
* Then, we add to the JSON of the configuration the name and default value of the environment variable
* Next, we instruct Chalice to automatically create a layer with the external libraries above. This will simplify our future API that will be able to reuse the layer.
* Next, we increase the memory size of Lambda from the default 128MB to 256MB
* Last, we write the modified configuration back to the file

In [13]:
import json
 
config_file = '.chalice/config.json'
with open(config_file) as json_file:
    app_config = json.load(json_file)
    
env_variable = {
    'planet': planet
}
app_config['stages']['dev']['environment_variables'] = env_variable
app_config['automatic_layer'] = True
app_config['stages']['dev']['lambda_memory_size'] = 256

with open(config_file, 'w') as outfile:
    json.dump(app_config, outfile)

In [14]:
!chalice deploy

Creating shared layer deployment package.
  Reusing existing shared layer deployment package.
Creating app deployment package.
Updating lambda layer: hello-planet-dev-managed-layer
Updating policy for IAM role: hello-planet-dev
Updating lambda function: hello-planet-dev
Updating rest API
Resources deployed:
  - Lambda Layer ARN: arn:aws:lambda:eu-west-1:672915487120:layer:hello-planet-dev-managed-layer:4
  - Lambda ARN: arn:aws:lambda:eu-west-1:672915487120:function:hello-planet-dev
  - Rest API URL: https://29c9xjwptk.execute-api.eu-west-1.amazonaws.com/api/


## Testing the API

In [15]:
import json
 
f = open('.chalice/deployed/dev.json',)
 
deployment_settings = json.load(f)
deployment_settings

{'resources': [{'name': 'managed-layer',
   'resource_type': 'lambda_layer',
   'layer_version_arn': 'arn:aws:lambda:eu-west-1:672915487120:layer:hello-planet-dev-managed-layer:4'},
  {'name': 'default-role',
   'resource_type': 'iam_role',
   'role_arn': 'arn:aws:iam::672915487120:role/hello-planet-dev',
   'role_name': 'hello-planet-dev'},
  {'name': 'api_handler',
   'resource_type': 'lambda_function',
   'lambda_arn': 'arn:aws:lambda:eu-west-1:672915487120:function:hello-planet-dev'},
  {'name': 'rest_api',
   'resource_type': 'rest_api',
   'rest_api_id': '29c9xjwptk',
   'rest_api_url': 'https://29c9xjwptk.execute-api.eu-west-1.amazonaws.com/api/'}],
 'schema_version': '2.0',
 'backend': 'api'}

In [16]:
rest_api_url = [
    resource['rest_api_url'] 
    for resource in deployment_settings['resources'] 
    if resource['name'] == 'rest_api'
][0]
rest_api_url

'https://29c9xjwptk.execute-api.eu-west-1.amazonaws.com/api/'

In [17]:
import requests

res = requests.get(f'{rest_api_url}')
res.json()

{'hello': 'EARTH with Mean Temperature (C) of 15'}

In [18]:
!curl $rest_api_url

{"hello":"EARTH with Mean Temperature (C) of 15"}

## Cleanup

At the end of the excercise we can clean up the resources

In [20]:
!chalice delete

Deleting Rest API: 29c9xjwptk
Deleting function: arn:aws:lambda:eu-west-1:672915487120:function:hello-planet-dev
Deleting IAM role: hello-planet-dev
Deleting layer version: arn:aws:lambda:eu-west-1:672915487120:layer:hello-planet-dev-managed-layer:4
